Программа повышения квалификации (научно-педагогических) работников НИУ ВШЭ

# Python для исследователей

*Автор: Татьяна Рогович, НИУ ВШЭ*  

## Web-scraping: сохраняем файлы

С помощью Python мы можем автоматически сохранять файлы. Например, на страничках сотрудников ВШЭ могут быть выложены резюме. 

https://www.hse.ru/org/persons/150045920

Давайте попробуем собрать адреса персональных страниц сотрудников факультета компьютерных наук и у тех сотрудников, у кого на странице есть файл с резюме, скачаем его.

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
link = 'https://cs.hse.ru/persons'
soup = BeautifulSoup(requests.get(link).text, 'lxml')

# находим класс, который отличает ссылки на персональные страницы сотрудников
persons = soup.find_all('a', {"class":"fa-person__name"}) 

# реконструируем ссылки и схораним кортеж - ссылка + ФИО сотрудника (и избавимся от дубликатов)
persons_links = [('http:' + a.get('href'), a.text.strip()) for a in set(persons)]
persons_links = sorted(persons_links, key=lambda x: x[1])

# в нашем примере будем работать с первыми десятью сотрудниками, 
# чтобы ускорить время исполнения кода
first_ten = persons_links[:10]
print(first_ten)

[('http://www.hse.ru/staff/abashidze', 'Абашидзе Хатуна Датоевна'), ('http://www.hse.ru/org/persons/348330900', 'Абовьян Наталья Михайловна'), ('http://www.hse.ru/org/persons/160951223', 'Аброскин Илья Дмитриевич'), ('http://www.hse.ru/org/persons/112929840', 'Авдеев Роман Сергеевич'), ('http://www.hse.ru/staff/avdoshin', 'Авдошин Сергей Михайлович'), ('http://www.hse.ru/org/persons/202722706', 'Аветисян Арутюн Ишханович'), ('http://www.hse.ru/org/persons/27235963', 'Агамирзян Игорь Рубенович'), ('http://www.hse.ru/org/persons/158485737', 'Айзенберг Антон Андреевич'), ('http://www.hse.ru/org/persons/224839518', 'Аксенов Сергей Андреевич'), ('http://www.hse.ru/org/persons/318659918', 'Аланов Айбек')]


Давайте напишем код, чтобы скачать резюме одного сотрудника.

In [3]:
person = first_ten[1]
page = requests.get(person[0]).text
print('Резюме' in page) # проверяем, есть ли на странице ссылка на резюме

True


In [4]:
# ссылка на резюме есть, поэтому продолжим обработку страницы
soup_link = BeautifulSoup(page, 'lxml')

# находим нужную ссылку по тексту
print(soup_link.find_all('a', text='Резюме')) 

# достаем сам адрес ссылки на резюме
print(soup_link.find_all('a', text='Резюме')[0].get('href')) 

# восстанавливаем полный адрес ссылки на резюме
cv_link = 'https://www.hse.ru' + soup_link.find_all('a', text='Резюме')[0].get('href')

# достаем имя сотрудника для того, чтобы назвать файл
name = person[1]
print(name)

# обращаемся по ссылке к файлу
doc = requests.get(cv_link)

# проверяем расширение файла, который хранится по ссылке
print(doc.headers['Content-Type'].split('/')[1] == 'pdf')


# создаем файл в режиме записи бинарной информации
with open(f'{name}.pdf', 'wb') as fh:
    # записываем в файл содержание файла по ссылке в байтах
    fh.write(doc.content)

[<a class="link" href="/data/2020/03/27/18380351003185/Abovyan_CV.pdf" target="_blank">Резюме</a>]
/data/2020/03/27/18380351003185/Abovyan_CV.pdf
Абовьян Наталья Михайловна
True


Теперь давайте напишем функцию и применим ее ко всем ссылкам в нашем списке.

In [5]:
def get_CV(person):
    page = requests.get(person[0]).text
    if 'Резюме' in page:
        soup_link = BeautifulSoup(page, 'lxml')
        cv_link = 'https://www.hse.ru' + soup_link.find_all('a', text='Резюме')[0].get('href')
        if doc.headers['Content-Type'].split('/')[1] == 'pdf':
            name = person[1]
            with open(f'{name}.pdf', 'wb') as fh:
                fh.write(doc.content)

In [6]:
for link in first_ten:
    get_CV(link)

## Самостоятельное задание: 
На странице сотрудников есть информация о владении иностранными языками. Для сотрудников факультета создайте словарь, где ключом будет язык, а значением - количество сотрудников, которые указали, что владеют им. Выведите ключи и значения словаря, отсортированные по значениям от большего к меньшему.

In [7]:
print(len(persons_links))

553


In [ ]:
import time
import random 

lang_dict = {}
cnt = 0

for link in persons_links:
    cnt += 1
    if cnt % 50 == 0:
        print(f'Обрабатываю ссылку №{cnt} из {len(persons_links)}')
    try:
        time.sleep(random.randint(1,5))
        page = requests.get(link[0]).text
        soup_link = BeautifulSoup(page, 'lxml')
        lang = soup_link.find_all('dl', {'class':"main-list large main-list-language-knowledge-level"})[0]
        for l in lang.find_all('dd'):
            l_text = l.get_text()
            lang_dict[l_text] = lang_dict.get(l_text, 0) + 1
    except:
        pass
        
print(sorted(lang_dict.items(), key = lambda x: x[0]), end='\n')